## Building NER model to identify the entities

In [91]:
import pandas as pd
import string

In [92]:
TRAIN_DATA = [
    ("Who is Shaka Khan?", {"entities": [(7, 17, "H0")]}),
    ("I like London and Berlin.", {"entities": [(7, 13, "H1"), (18, 24, "H0")]}),
]

In [93]:
data = pd.read_csv("sent_classifier_train.csv", index_col=False)

In [172]:
#Preparing Training Data for Spacy
#table = str.maketrans('[]','')
TRAIN_DATA=[]
for index, row in data.iterrows():
    #print(row['Spacy'])
    entities_dict={}
    
    sentence=row['Sentence']
    entities=row['Spacy'].split("],")
    entities_list=[]
    if(len(row['Spacy'].strip())>2):
        for entity_str in entities:
            #entity.translate(table)
            req_entity_str=entity_str.replace('[','').replace(']','')
            req_entity=req_entity_str.split(',')
            start_index=int(req_entity[0].strip())
            #print(start_index)
            end_index=int(req_entity[1].strip())
            #print(end_index)
            entity=req_entity[2].strip().replace("'","")
            #print(entity)
            entities_list.append((start_index,end_index,entity))
    entities_dict['entities']=entities_list
    TRAIN_DATA.append((row['Sentence'],entities_dict))
    
        


In [173]:
TRAIN_DATA[0]

('Isaac David Abella (June 20, 1934 – October 23, 2016) was a Professor of Physics at The University of Chicago. ',
 {'entities': [(0, 4, 'H0'),
   (6, 10, 'H0'),
   (12, 17, 'H0'),
   (25, 28, 'H1'),
   (29, 33, 'H1'),
   (34, 35, 'H1'),
   (36, 43, 'H1'),
   (44, 47, 'H1'),
   (48, 53, 'H1'),
   (54, 57, 'H1'),
   (60, 69, 'H1'),
   (70, 72, 'H1'),
   (73, 80, 'H1'),
   (81, 83, 'H1'),
   (84, 87, 'H1'),
   (88, 98, 'H1'),
   (99, 101, 'H1'),
   (102, 110, 'H1')]})

## Code for Spacy Model

In [174]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

In [59]:
@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    new_model_name=("New model name for model meta.", "option", "nm", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)

SyntaxError: unexpected EOF while parsing (<ipython-input-59-34251adb9d5b>, line 6)

In [177]:
def main(model=None, new_model_name="Privacy_Redaction", output_dir="Spacy_Model", n_iter=100):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")
        
    #Add H0 and H1 as entities
    ner.add_label("H0") 
    ner.add_label("H1") 
    
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
        
    move_names = list(ner.move_names)
    
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    
    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                #print(batch)
                texts, annotations = zip(*batch)
                #print(zip(*batch))
                #print(text)
                #print(annotationsations)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)
    
    # test the trained model
    test_text = "Do you like horses?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)
        
    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

In [178]:
if __name__ == "__main__":
    main()

Created blank 'en' model
Losses {'ner': 7001.339179119419}
Losses {'ner': 5274.288624803818}
Losses {'ner': 4233.215686360598}
Losses {'ner': 3708.6963201080725}
Losses {'ner': 3470.051681057848}
Losses {'ner': 2994.283204119511}
Losses {'ner': 2580.1443003455224}
Losses {'ner': 2420.2402061846124}
Losses {'ner': 2193.045202154296}
Losses {'ner': 2040.6577171647307}
Losses {'ner': 1962.9975720766047}
Losses {'ner': 1784.862245879911}
Losses {'ner': 1586.1338208529787}
Losses {'ner': 1539.7148191993008}
Losses {'ner': 1516.195417077236}
Losses {'ner': 1454.382354292505}
Losses {'ner': 1391.2223248088128}
Losses {'ner': 1277.1871288780574}
Losses {'ner': 1164.4331448397218}
Losses {'ner': 1169.8963033545901}
Losses {'ner': 1135.1620668712903}
Losses {'ner': 1076.4378280924664}
Losses {'ner': 1085.606820821391}
Losses {'ner': 1005.0197114568757}
Losses {'ner': 1024.7630288457262}
Losses {'ner': 991.8002231639651}
Losses {'ner': 944.9877999481828}
Losses {'ner': 926.660604939327}
Losses {'

In [179]:
output_dir="Spacy_Model"
nlp2 = spacy.load(output_dir)
nlp = spacy.load("en_core_web_sm")

In [211]:
sample="He became a mugham teacher in 1973 at the Abilov Culture House in Baku, and also worked as a teacher at the Zeynally Music College in 1977."
doc1 = nlp2(sample)
doc2=nlp(sample)

In [212]:
print("My model")
for ent in doc1.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

My model
mugham 12 18 H1
teacher 19 26 H1
in 27 29 H1
1973 30 34 H1
at 35 37 H1
the 38 41 H1
Abilov 42 48 H1
Culture 49 56 H1
House 57 62 H1


In [210]:
print("Spacy model")
for ent in doc2.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Spacy model
Aghakhan Abdullayev 0 19 PERSON
Azerbaijani 21 32 NORP
Abdullayev 42 52 PERSON
February 1950 56 69 DATE
25 December 2016 74 90 DATE
Azerbaijani 99 110 NORP


In [126]:
test="Abdullayev was born in Baku where he graduated from secondary school in 1968, then continued his education at the Zeynally Music College until 1973."

In [127]:
test[23:27]

'Baku'